In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import KFold

In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned_final.csv')
test = pd.read_csv(path + 'test_cleaned_final.csv')

In [3]:
### load training index
indexfile = '/home/kai/data/kaggle/talkingdata/wl/data/trainset/train_index_all_shuffle.npy'
train_index = np.load(indexfile)
### load validation index
indexfile = '/home/kai/data/kaggle/talkingdata/wl/data/trainset/val_index_all_shuffle.npy'
val_index = np.load(indexfile)



### load feature cols
import json as js
featurefile = '/home/kai/data/kaggle/talkingdata/wl/data/features/feature_cols.json'
with open(featurefile, 'r') as myjs:
    added_features = js.load(myjs)

In [4]:
added_features

['ip_count',
 'ip_minute_count',
 'app_channel_count',
 'ip_app_count',
 'ip_app_hour_count',
 'ip_second_count',
 'ip_app_os_count',
 'ip_device_hour_count',
 'ip_os_hour_count',
 'ip_day_hour_count',
 'ip_channel_count',
 'app_channel_day_count',
 'ip_device_count',
 'ip_day_count',
 'ip_hour_count',
 'ip_mean',
 'app_os_channel_mean',
 'hour_minute_second_mean',
 'app_os_hour_mean',
 'ip_second_mean',
 'ip_app_device_mean',
 'device_minute_second_mean',
 'ip_app_mean',
 'ip_channel_mean',
 'device_minute_mean',
 'app_channel_hour_mean',
 'app_device_os_mean',
 'device_os_hour_mean',
 'ip_app_channel_mean',
 'channel',
 'app',
 'os',
 'device',
 'second',
 'minute',
 'hour',
 'ip']

# Get Historical data

In [5]:
history_index = list(set(range(len(train))) - set(train_index) - set(val_index))
df_history = train.iloc[history_index].copy()
df_train = train.iloc[train_index].copy()
df_val = train.iloc[val_index].copy()


df_all = pd.concat([df_train, df_val, test])
train_length = len(df_train)
val_length = len(df_val)
test_length = len(test)

import gc
del train
del test
del df_train
del df_val
gc.collect()

40

In [6]:
import sys
print(sys.getsizeof(df_all)/ 1024 **3)
print(len(df_all))

6.597382061183453
73790469


In [7]:
df_all.columns

Index(['app', 'channel', 'day', 'device', 'hour', 'ip', 'is_attributed',
       'minute', 'os', 'second', 'timestamp'],
      dtype='object')

In [8]:
orders = {}
feature_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'day',
              'hour',
              'minute',
              'second']

# feature_col = ['ip', 
#               'app', 
#               'device', 
#               'os', 
#               'channel']
for col in feature_col:
    orders[col] = 10 ** (int(np.log(max(df_history[col].max(),df_all[col].max() ) + 1) / np.log(10)) + 1)
def get_group(df, cols):
    """
    define an encoding method which can ganrantee the adding value will be unique.
    eg: artist_name_composer will be a combination of (artist_name,composer) and the encoding will reflect the unqiue combination of those two
    """
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
        
    return group

import gc
# del train
gc.collect()

0

In [9]:
orders

{'app': 1000,
 'channel': 1000,
 'day': 100,
 'device': 10000,
 'hour': 100,
 'ip': 1000000,
 'minute': 100,
 'os': 1000,
 'second': 100}

# count
plan 1. count from historical data  
plan 2. count from all data

In [10]:
def count(df_history, df_train, cols, target=None):
    """
    Purpose: add a new feature to training df.count the number of records for each feature combination (eg, artist_name_composer)) 
    """
    
    group = get_group(df_train, cols)
    group_all = get_group(df_history, cols)
    
    count_map = group_all.value_counts()
    
    return group.map(count_map).fillna(0)

# mean
mean P(target | feature combination)

purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer))
Get the conditional Probability only from historical data and apply to train data.

P(replay | X feature combination) = P( replay & X feature combination) / P (X feature combination)  
=(count(replay & X feature combination) / count(total)) / (count(X feature combination) / count(total))  
= count(replay & X feature combination) / count(X feature combination)  
= sum((replay & X feature combination)) / count(X feature combination)  
= sum((replay or not replayed & X feature combination)) / count(X feature combination)# since replay is 1, not replay is 0  
= sum( X feature combination) / count(X feature combination)  
= mean(X feature combination)  

In [11]:
def scaller(num):
    sca = 1
    while num * sca < 1:
        sca *= 10
    return sca

def mean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  
    # encoding df's cols into a new series
    group = get_group(df_train, cols)
    # encoding df_history's cols into a new series
    group_history = get_group(df_history, cols)
    # get the conditional probability p(target| feature combination. eg, artist_name_composer) 
    mean_map = df_history.groupby(group_history)[target].mean()
    # mean_map: key - encoding, value - target mean
#     ### sca
#     m_min = mean_map[mean_map > 0].min()
#     sca = scaller(m_min)
#     mean_map *= sca
#     ###

    return group.map(mean_map).fillna(-1)

# reversemean
reverse mean P(feature combination | target)

In [12]:
def reversemean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  
    # encoding df's cols into a new series
    group = get_group(df_train, cols)
    # encoding df_history's cols into a new series
    group_history = get_group(df_history, cols)
    # get the conditional probability p(target| feature combination. eg, artist_name_composer) 
    positive = group_history[df_history[target] == 1]
    negative = group_history[df_history[target] == 0]
    index_p = set(positive.unique())
    index_n = set(negative.unique())
    index_n.difference_update(index_p)
    map_reverse_p = positive.groupby(positive).count() / len(positive)
    map_reverse_n = pd.Series(np.zeros(len(index_n)), index=index_n)
    map_reverse = pd.concat([map_reverse_p, map_reverse_n])
    return group.map(map_reverse).fillna(-1)

# generate all cols

In [13]:
from itertools import combinations


combine_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel', 
              'day',
              'hour',
              'minute',
              'second']

# combine_col = ['ip', 
#               'app', 
#               'device', 
#               'os', 
#               'channel']

def col_name(cols, func=None):
    if func is None:
        return '_'.join(cols)
    else:
        return '_'.join(cols) + '_' + func.__name__

counter = 0



exception_list = []
for num_col in [1,2,3]:
    for cols in combinations(combine_col, num_col):
#         for func in [count, mean]:
        for func in [count, mean]:
            feature_name = col_name(cols, func=func)
            if feature_name in added_features:
                counter += 1
                if func.__name__ == count.__name__:
                    print('count function')
                    df_all[feature_name] = func(df_all, df_all, cols, target='is_attributed')
                else:
                    print('mean function')
                    df_all[feature_name] = func(df_history, df_all, cols, target='is_attributed')
                all_str = 'all {}:   {}   \t\t\t size: {} G.'.format(counter, feature_name, sys.getsizeof(df_all)/ 1024 **3)
                print(all_str)
                with open('feature_all.txt', 'w') as text_file:
                    text_file.write(all_str + '\n')


count function
all 1:   ip_count   			 size: 7.147163897752762 G.
mean function
all 2:   ip_mean   			 size: 7.696945734322071 G.
count function
all 3:   ip_app_count   			 size: 8.24672757089138 G.
mean function
all 4:   ip_app_mean   			 size: 8.79650940746069 G.
count function
all 5:   ip_device_count   			 size: 9.346291244029999 G.
count function
all 6:   ip_channel_count   			 size: 9.896073080599308 G.
mean function
all 7:   ip_channel_mean   			 size: 10.445854917168617 G.
count function
all 8:   ip_day_count   			 size: 10.995636753737926 G.
count function
all 9:   ip_hour_count   			 size: 11.545418590307236 G.
count function
all 10:   ip_minute_count   			 size: 12.095200426876545 G.
count function
all 11:   ip_second_count   			 size: 12.644982263445854 G.
mean function
all 12:   ip_second_mean   			 size: 13.194764100015163 G.
count function
all 13:   app_channel_count   			 size: 13.744545936584473 G.
mean function
all 14:   device_minute_mean   			 size: 14.2943277731537

# Saving Files

In [14]:
target='is_attributed'
train_col = added_features.copy()
train_col.append(target)
train = df_all.iloc[:train_length][train_col]
val = df_all.iloc[train_length:train_length+val_length][train_col]
test = df_all.iloc[train_length+val_length:][added_features]
print(len(test))

18790469


In [15]:
train.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/train_countAndmean_index_all_shuffle_0405.csv', index=False)
print('training saving done!')
val.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/val_countAndmean_index_all_shuffle_0405.csv', index=False)
print('val saving done!')
test.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/test_countAndmean_index_all_shuffle_0405.csv', index=False)
print('testing saving done!')

training saving done!
val saving done!
testing saving done!


In [21]:
print(len(test))

18790469
